In [ ]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.image as mpimg
from skimage import io
import importDataSet
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
print(tf.__version__)

In [ ]:
# location = "/media/yigit/YEDEK320/final_dataset/"  # Location of the dataset
# locationSSD = "/media/yigit/2E58D09A58D0625F/tf"

location = "D:\\final_dataset\\"
locationSSD = "C:\\tf"
itemNumbers=[0,0,0]
totalNumberOfImages = 0;
nameDataset = {}

In [ ]:
verbose = 1


In [ ]:
totalNumberOfImages, nameDataset, itemNumbers = importDataSet.findTheFiles(location=location,verbose=0)



In [ ]:
importDataSet.showSomeSamples(nameDataset)
if verbose == 1:
    items = ["pipe","cable","diver"]
    print("There are total {0} images in the dataset".format(totalNumberOfImages))
    for i, item in enumerate(itemNumbers):
        print("There are {0} {1}-containing image".format(item, items[i]))

    # for item in nameDataset.keys():
    #     print(item + "-->")
    #     print(nameDataset[item])



If the tfrecord file is not created, comment out this sction to create the tf record.

In [ ]:
#file_list = importDataSet.writeTfrecords(nameDataset, location+"tf/", how_many_files=totalNumberOfImages, verbose= 0)

Just to check if the file is created correctly

In [ ]:
# imageBatch,labelBatch = importDataSet.getBatchFromFile(location+"tf/",n_batch=200,sess=sess)

# print(imageBatch.shape,labelBatch.shape)
# importDataSet.showObtainedImages(imageBatch,  labelBatch)

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
with tf.Session() as sess1:
    imageSet, labels = importDataSet.importAllDataSet(locationSSD, totalNumberOfImages, session=sess1)

In [ ]:
print(imageSet.shape,labels.shape)
print(np.result_type(imageSet))

In [ ]:
trainSet, trainLabels = [imageSet[0:int(imageSet.shape[0]*0.8)], labels[0:int(labels.shape[0]*0.8)]]
print(trainSet.shape, trainLabels.shape)
trainSize = trainSet.shape[0]
validationSet, validationLabels = [imageSet[int(imageSet.shape[0]*0.8):], labels[int(labels.shape[0]*0.8):]]
print(validationSet.shape, validationLabels.shape)
validationSize = validationSet.shape[0]

In [ ]:
import weaklySupervisedObjectDetectionNetwork as WSODN

In [ ]:

# print(batch_index)
# print(trainSet[batch_index].shape)
# print(trainLabels[batch_index].shape)
error_list =[]
with tf.Session() as sess2:
    WSCNN = WSODN.WSCNN("./vgg16_weights.npz",sess2)  # initialize the network
    
    loss = tf.reduce_sum(1+tf.exp(-1*WSCNN.labels*WSCNN.overAllScore))
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-10).minimize(loss)
    
    init = tf.initialize_all_variables()
    sess2.run(init)
    for iteration in range(1000):
        batch_index = np.random.choice(trainSize, 4)
        _, asd = sess2.run([optimizer,loss],feed_dict={WSCNN.batch:trainSet[batch_index], WSCNN.labels:trainLabels[batch_index]})
        print(batch_index, asd)
        error_list.append(asd)